<a href="https://colab.research.google.com/github/digital0923RJ/Kaggle-Project/blob/main/Rohlik_Sales_Forecasting_Challenge_(Ranked_142_out_of_838_Participants).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Rohlik Sales Forecasting Challenge**

### **About Rohlik Group**
Rohlik Group, a leading European e-grocery innovator, is revolutionizing the food retail industry. We operate across **11 warehouses** in **Czech Republic, Germany, Austria, Hungary, and Romania**.

### **Goal**
The challenge focuses on **predicting the sales** of each selected warehouse inventory for the next **14 days** using **historical sales data**.

### **Evaluation Metric**
Submissions are evaluated using **Weighted Mean Absolute Error (WMAE)** between the predicted sales and the actual sales. Weights for the test evaluation are provided in the **Data section**.

---

## **My Approach**

### **1️⃣ Addressing Residuals in Time-Series Forecasting**
- Since this is a **time-series problem**, I focused on **lag features** and **residual analysis**.
- Sorted the dataset by **warehouse names, dates, and sales** to detect trends.
- Observed that **residuals were increasing over time**, requiring further investigation.
- To address this, I **filled in missing dates, holidays**, and introduced **lagging features**.

### **2️⃣ Skewed Distribution and Log Transformation**
- The initial model’s **prediction distribution was highly skewed**, making accurate prediction difficult.
- Applied **log transformation** to make the target variable follow a **normal distribution**, which stabilized the model’s predictions.
- Removed **negative residual values**, as they negatively impacted **WMAE scoring**.

### **3️⃣ Hyperparameter Optimization Using Optuna**
- Used **Optuna** to tune key hyperparameters such as:
  - **Learning rate**
  - **Number of leaves**
  - **Max depth**
  - **Regularization (L1, L2)**
- The optimization was based on **TimeSeriesSplit cross-validation** and evaluated using **Weighted MAE (WMAE)**.

### **4️⃣ Model Training and Submission**
- Trained a **LightGBM (LGBM) model** using the best hyperparameters.
- Applied **inverse log transformation** to convert predictions back to the original scale.
- Ensured **no negative values** in the final predictions before submission.


---


In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder
import numpy as np
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
base_path = "/content/drive/MyDrive/포트폴리오/rohlik-sales-forecasting-challenge-v2/"
train = pd.read_csv(f'{base_path}sales_train.csv', parse_dates=['date'])
inventory = pd.read_csv(f'{base_path}inventory.csv')
test = pd.read_csv(f'{base_path}sales_test.csv', parse_dates=['date'])
calendar = pd.read_csv(f'{base_path}calendar.csv', parse_dates=['date'])
test_weights = pd.read_csv(f'{base_path}test_weights.csv')
calendar_enriched = pd.read_csv(f'{base_path}calendar_enriched.csv')

In [ ]:
train[['total_orders', 'sales']] = train[['total_orders', 'sales']].fillna(train[['total_orders', 'sales']].mean(numeric_only=True))

In [ ]:
import pandas as pd

def filter_train_by_test_unique_id(sales_train, sales_test, unique_id_col='unique_id'):

    # get the unique_id from test dataset
    test_unique_ids = set(test[unique_id_col].unique())

    # leave only unique_id from test data from the train data
    filtered_train_df = train[train[unique_id_col].isin(test_unique_ids)].copy()

    # Before and After of filtering of 'unique_id'
    train_unique_count_before = train[unique_id_col].nunique()
    train_unique_count_after = filtered_train_df[unique_id_col].nunique()
    test_unique_count = test[unique_id_col].nunique()

    print(f"🔹 Before filtering Train_data Unique ID #N : {train_unique_count_before}")
    print(f"🔹 After filtering Train_data Unique ID #N: {train_unique_count_after}")
    print(f"🔹 Test data Unique ID #N: {test_unique_count}")

    return filtered_train_df

# filtering
filtered_sales_train = filter_train_by_test_unique_id(train, test, unique_id_col='unique_id')


🔹 필터링 전 Train 데이터 Unique ID 개수: 5390
🔹 필터링 후 Train 데이터 Unique ID 개수: 3625
🔹 Test 데이터 Unique ID 개수: 3625


In [ ]:
train = filtered_sales_train

In [ ]:
# Ensure 'date' columns are of datetime type before merging
train['date'] = pd.to_datetime(train['date'])
calendar_enriched['date'] = pd.to_datetime(calendar_enriched['date'])

# Now perform the merge
train = train.merge(calendar_enriched, on=['warehouse', 'date'], how='left')
test = test.merge(calendar_enriched, on=['warehouse', 'date'], how='left')

In [ ]:
train=train.merge(test_weights,on='unique_id',how='left')

In [ ]:
train=train.merge(inventory,on=['warehouse','unique_id'],how='left')
test=test.merge(inventory,on=['warehouse','unique_id'],how='left')

train=train[train['date']>='2021-06-01']
train.head()

,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,...,date_second_closed_day,date_day_after_two_closed_days,date_year,weight,product_unique_id,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en
0,4845,2024-03-10,Budapest_1,6436.0,16.34,646.26,1.00,0.00000,0.0,0.0,...,0,0,2024,1.925596,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1
2,4845,2021-12-20,Budapest_1,6507.0,34.55,455.96,1.00,0.00000,0.0,0.0,...,0,0,2021,1.925596,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1
3,4845,2023-04-29,Budapest_1,5463.0,34.52,646.26,0.96,0.20024,0.0,0.0,...,0,0,2023,1.925596,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1
4,4845,2022-04-01,Budapest_1,5997.0,35.92,486.41,1.00,0.00000,0.0,0.0,...,0,0,2022,1.925596,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1
5,4845,2024-03-02,Budapest_1,6760.0,27.26,646.26,1.00,0.00000,0.0,0.0,...,0,0,2024,1.925596,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1


In [ ]:
train = train.drop(columns=['availability'])
train.dropna(subset=['sales'], inplace=True)

In [ ]:
test['sales'] = 0
df = pd.concat([train, test], ignore_index=True).sort_values('date')

In [ ]:
PERIODS = [14, 20, 28, 35, 90, 180, 270]
def add_date_features(df):
    df['date_month'] = df['date'].dt.month
    df['date_day'] = df['date'].dt.day
    df['date_weekofyear'] = df['date'].dt.isocalendar().week
    df['date_weekday'] = df['date'].dt.weekday
    df['date_year'] = df['date'].dt.dayofyear
    df['date_year_sin'] = np.sin((df['date_year'] - df['date_year'].min()) / (df['date_year'].max() - df['date_year'].min()) * 2 * np.pi)
    df['date_year_sin'] = np.sin(df['date_year'] / 1 * 2 * np.pi)
    df['date_month_sin'] = np.sin(df['date_month'] / 12 * 2 * np.pi)
    return df

def add_product_category(df):
    df['category'] = df['name'].str.split('_',expand=True)[0]
    return df

def add_lagged_product_sales(df):
    for shift in PERIODS:
        df[f'product_sales_{shift}']=df.groupby(['warehouse','name'])['sales'].shift(periods=shift)
    return df

df = add_date_features(df)
df = add_product_category(df)
df = add_lagged_product_sales(df)

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['weekday'] = df['date'].dt.weekday
df['dayofweek'] = df['date'].dt.dayofweek
df['weekofyear'] = df['date'].dt.isocalendar().week
df['dayofyear'] = df['date'].dt.dayofyear
df['is_month_start'] = df['date'].dt.is_month_start
df['is_month_end'] = df['date'].dt.is_month_end
df['quarter'] = df['date'].dt.quarter
df["total_dic"]=df['type_0_discount']+df['type_0_discount']+df['type_1_discount']+df['type_2_discount']+df['type_3_discount']+df['type_4_discount']+df['type_5_discount']+df['type_6_discount']
df['total_orders_']=df['total_orders']/df['sell_price_main']
df['total_orders_dic']=df['total_orders_']/df["total_dic"]
df['total_orders_sell_price_main']=df['sell_price_main']/df["total_dic"]
for i in range(7):
    df[f'total_orders{i}']=df[f'type_{i}_discount']/df["total_orders"]
    df[f'total_orders_sell_price_main_{i}']=df[f'type_{i}_discount']/df["total_orders_sell_price_main"]
    df[f'sell_price_main{i}']=df[f'type_{i}_discount']/df["sell_price_main"]
    df[f'sell_price_main_x_{i}']=df[f'type_{i}_discount']/(df["sell_price_main"]*df["total_orders"])
    df[f'total_orders_dic{i}']=df[f'type_{i}_discount']/df["total_orders_dic"]

    df[f'_total_orders{i}']=df[f'type_{i}_discount']*df["total_orders"]
    df[f'_total_orders_sell_price_main_{i}']=df[f'type_{i}_discount']*df["total_orders_sell_price_main"]
    df[f'_sell_price_main{i}']=df[f'type_{i}_discount']*df["sell_price_main"]
    df[f'_total_orders_dic{i}']=df[f'type_{i}_discount']*df["total_orders_dic"]

df.fillna(0, inplace=True)

categorical_columns=['unique_id']+list(df.select_dtypes("object").columns)

for col in categorical_columns:
    df[col] = df[col].astype('category')

In [ ]:
train_start_date  = '2020-08-01'
train_end_date  = '2024-06-02'
test_start_date = '2024-03-18'
test_end_date = '2024-06-01'
train_df = df[(df['date'] >= train_start_date) & (df['date'] <= train_end_date)]
test_df  = df[(df['date'] >  train_end_date)]
train_data = df[(df['date'] < train_end_date)]
test_data = df[(df['date'] >= test_start_date)]

In [ ]:
train_df.shape, test_df.shape, train_data.shape, test_data.shape

((2938869, 122), (47021, 122), (2935551, 122), (295084, 122))

In [ ]:
X = df.drop(['sales', 'date','weight'], axis=1)
y = np.log1p(df['sales'])
train_weights = train_data['weight']

In [ ]:
X_train = train_data.drop(['sales', 'date', 'weight'], axis=1)
y_train = np.log1p(train_data['sales'])
train_weights = train_data['weight']
cols=X.select_dtypes(["int","float"]).columns

In [ ]:
X_train.shape, y_train.shape, train_weights.shape

((2935551, 119), (2935551,), (2935551,))

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.1 MB/s eta 0:00:00


In [ ]:
from sklearn.preprocessing import RobustScaler
import numpy as np

sc = RobustScaler()

for col in cols:
    df[col].replace([np.inf, -np.inf], df[col].min(), inplace=True)
    df[col].fillna(X_train[col].mean(), inplace=True)
    # Transform the entire column in 'df' instead of just 'X_train'
    df[col] = sc.fit_transform(df[[col]])

In [ ]:
categorical_feature_indices = [X.columns.get_loc(col) for col in categorical_columns if col in X.columns]

In [ ]:
from lightgbm import log_evaluation, early_stopping

In [ ]:
X_train = train_data.drop(['sales', 'date', 'weight'], axis=1)
y_train = np.log1p(train_data['sales'])
train_weights = train_data['weight']

X_test = test_data.drop(['sales', 'date', 'weight'], axis=1)
y_test = np.log1p(test_data['sales'])

test_weights = test_data['weight']

In [ ]:
cols=X.select_dtypes(["int","float"]).columns

In [ ]:
import optuna
import lightgbm as lgb
import numpy as np
from sklearn.metrics import mean_absolute_error

def objective(trial):
    """
    Optuna 최적화 함수: LightGBM 모델의 하이퍼파라미터를 튜닝.
    """
    # Hyperparameter
    params = {
        'objective': 'regression',
        'metric': 'mae',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 5, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-5, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-5, 10.0),
        'num_boost_round': trial.suggest_int('num_boost_round', 500, 5000),
        'verbose': -1
    }

    # LightGBM dataset
    train_dataset = lgb.Dataset(X_train, label=y_train,
                                categorical_feature=categorical_feature_indices,
                                weight=train_weights)
    valid_dataset = lgb.Dataset(X_test, label=y_test,
                                categorical_feature=categorical_feature_indices,
                                weight=test_weights,
                                reference=train_dataset)

    # model training
    model = lgb.train(params, train_dataset,
                      num_boost_round=params['num_boost_round'],
                      valid_sets=[valid_dataset])


    # model prediction
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    # Reverse transformation from log
    y_test_original = np.expm1(y_test)
    y_pred_original = np.expm1(y_pred)

    # Weighted MAE
    weighted_mae = np.sum(test_weights * np.abs(y_test_original - y_pred_original)) / np.sum(test_weights)

    return weighted_mae

# Optuna implementation
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

print("Best trial:")
print(study.best_trial.params)

In [ ]:
best_params = {
    'learning_rate': 0.07197210595947787,
    'num_leaves': 123,
    'max_depth': 15,
    'min_child_samples': 30,
    'subsample': 0.8486600816122877,
    'colsample_bytree': 0.9205837051511344,
    'reg_alpha': 1.8968996438676256e-05,
    'reg_lambda': 0.00026096880620833136,
    'num_boost_round': 2697
}

In [ ]:
sub = test_df.copy()
final_train_dataset = lgb.Dataset(X, label=y,
                                  categorical_feature=categorical_feature_indices,
                                  weight=df['weight'])

In [ ]:
test_df.columns

Index(['unique_id', 'date', 'warehouse', 'total_orders', 'sales',
       'sell_price_main', 'type_0_discount', 'type_1_discount',
       'type_2_discount', 'type_3_discount',
       ...
       '_total_orders_dic5', 'total_orders6', 'total_orders_sell_price_main_6',
       'sell_price_main6', 'sell_price_main_x_6', 'total_orders_dic6',
       '_total_orders6', '_total_orders_sell_price_main_6',
       '_sell_price_main6', '_total_orders_dic6'],
      dtype='object', length=122)

In [ ]:
final_model = lgb.train(best_params,
                        train_set=final_train_dataset)

# Drop 'date' and 'weight' columns from test_df before prediction
test_df_processed = test_df.drop(['sales', 'date', 'weight'], axis=1)
final_y_pred = final_model.predict(test_df_processed)
sub['sales_hat'] = np.expm1(final_y_pred)
sub['id']= sub['unique_id'].astype(str) + "_" + sub['date'].astype(str)
sub[['id','sales_hat']].to_csv("submission.csv",index=False)


[LightGBM] [Warning] Unknown parameter: learning_rates
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Warning] Unknown parameter: learning_rates
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.605215 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 21959
[LightGBM] [Info] Number of data points in the train set: 2985890, number of used features: 109
[LightGBM] [Info] Start training from score 3.552977


In [ ]:
sub['sales_hat'] = sub['sales_hat'].clip(lower=0)
sub[['id','sales_hat']].to_csv("submissio_sss1.csv", index=False)